In [1]:
!pip install snowflake-snowpark-python snowflake-ml-python snowflake.core -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dask 2024.11.2 requires cloudpickle>=3.0.0, but you have cloudpickle 2.2.1 which is incompatible.
distributed 2024.11.2 requires cloudpickle>=3.0.0, but you have cloudpickle 2.2.1 which is incompatible.


In [2]:
!pip install trulens-core trulens-providers-cortex trulens-connectors-snowflake llama-index llama-index-embeddings-huggingface llama-index-readers-github snowflake-sqlalchemy -q

In [2]:
from snowflake.snowpark.session import Session
import os
import json



connection_params = {
  "account": SNOWFLAKE_ACCOUNT,
  "user": SNOWFLAKE_USER,
  "password": SNOWFLAKE_USER_PASSWORD,
  "role": SNOWFLAKE_ROLE,
  "database": SNOWFLAKE_DATABASE,
  "schema": SNOWFLAKE_SCHEMA,
  "warehouse": SNOWFLAKE_WAREHOUSE
}

# Create the Snowpark session
snowpark_session = Session.builder.configs(connection_params).create()

ModuleNotFoundError: No module named 'snowflake'

In [3]:
import pandas as pd

# Load CSV
csv_file = "/kaggle/input/indian-supreme-court-judgments/judgments.csv"
metadata_df = pd.read_csv(csv_file)

# Preview data
print(metadata_df.head())


   diary_no Judgement_type                          case_no  \
0    5-2021              J                 MA-000083 - 2021   
1   28-2021              J    C.A. No.-000867-000867 - 2021   
2   70-2020              J  Crl.A. No.-000167-000167 - 2021   
3   82-2021              J         SLP(C) No.-000811 - 2021   
4  114-2021              J    C.A. No.-000390-000390 - 2021   

                                                 pet  \
0                                     UNION OF INDIA   
1  INDEX MEDICAL COLLEGE HOSPITAL AND RESEARCH CE...   
2                                       ARCHANA RANA   
3          THE EMPLOYEES STATE INSURANCE CORPORATION   
4  KALOJI NARAYANA RAO UNIVERSITY OF HEALTH SCIENCES   

                                                 res  \
0  ASSOCIATION OF UNIFIED TELECOM SERVICEPROVIDER...   
1                        THE STATE OF MADHYA PRADESH   
2                         THE STATE OF UTTAR PRADESH   
3                              M/S. TEXMO INDUSTRIES   
4   

In [4]:
# Fix URLs in the metadata
base_url = "https://main.sci.gov.in/jonew/"
metadata_df['temp_link'] = base_url + metadata_df['temp_link'].fillna("")

In [ ]:
metadata_df

In [5]:
!pip install pymupdf -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 73.4 MB/s eta 0:00:00:00:0100:01


In [7]:
!pip install easyocr -q


In [ ]:
!pip install pdfplumber

In [8]:
!pip install pypdf2 -q

In [15]:
!pip install tqdm

In [13]:
!apt-get update
!apt-get install poppler-utils

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease         
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease                                              
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease                                    
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease                                      
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease                                    
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease                                          
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease     
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/so

In [ ]:
pip install dask-cudf-cu12

In [28]:
import PyPDF2

def is_valid_pdf(pdf_path):
    try:
        with open(pdf_path, 'rb') as f:
            reader = PyPDF2.PdfReader(f)
            if len(reader.pages) > 0:
                return True
            else:
                return False
    except Exception as e:
        #print(f"Invalid PDF file or error: {e}")
        return False


In [6]:
import os
import easyocr
import fitz  # PyMuPDF
from pdf2image import convert_from_path
import cudf
import concurrent.futures
from time import time

In [ ]:
# Function to extract text with OCR or from machine-readable text
def extract_text_with_easyocr(pdf_path):
    try:
        # Try extracting machine-readable text using PyMuPDF
        doc = fitz.open(pdf_path)
        text = ""
        for page_num in range(doc.page_count):
            page = doc.load_page(page_num)
            text += page.get_text()  # Extract text from each page
        
        if text.strip():  # If text is found, return it
            #print(f"Normal Extraction Done! {pdf_path}")
            return text.strip(), False  # No OCR used, machine-readable text found
        else:
            # Fall back to OCR if no machine-readable text
            #print(f"Machine-readable text not found in {pdf_path}, using OCR.")
            reader = easyocr.Reader(['en'], gpu=True)  # GPU-enabled OCR
            images = convert_from_path(pdf_path)
            
            ocr_text = ""
            for image in images:
                result = reader.readtext(image)
                ocr_text += " ".join([text[1] for text in result])

            return str(ocr_text.strip()), True  # OCR processed with easyOCR
    
    except Exception as e:
        #print(f"Error processing {pdf_path}: {e}")
        return None, None

In [ ]:
t = extract_text_with_easyocr('/kaggle/input/indian-supreme-court-judgments/pdfs/1615-2004___jonew__judis__20959.pdf')

In [25]:
# Function to process PDFs and return a dictionary of extracted text
def process_pdf(pdf_path):
    text, ocr_used = extract_text_with_easyocr(pdf_path)
    if text:
        return {"File Name": os.path.basename(pdf_path), "Extracted Text": text}
    return None


In [4]:
import os
import concurrent.futures
import cudf
import pandas as pd
from tqdm import tqdm

In [ ]:
# Function to process multiple PDFs in parallel and save results in a CUDA DataFrame
def process_pdfs_in_folder(folder_path, output_file):
    # Get list of all PDFs in the folder
    pdf_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(".pdf")]
    
    # Initialize the list to store results
    results = []
    
    # Display progress bar with total files
    with tqdm(total=len(pdf_files), desc="Processing PDFs", unit="file") as pbar:
        # Use ThreadPoolExecutor for multi-threading
        with concurrent.futures.ThreadPoolExecutor(max_workers=8) as executor:
            # Map the process_pdf function to each PDF file
            future_to_pdf = {executor.submit(process_pdf, pdf): pdf for pdf in pdf_files}
            
            # Collect results as they are completed
            for future in concurrent.futures.as_completed(future_to_pdf):
                result = future.result()
                if result:
                    results.append(result)  # Append the result
                pbar.update(1)  # Update the progress bar for each completed task

    # Create a CUDA DataFrame from the results
    if results:
        df = cudf.DataFrame.from_pandas(pd.DataFrame(results))
        # Save to CSV (or other formats) using GPU-accelerated saving
        df.to_csv(output_file, index=False)
        print(f"Extraction completed successfully. Results saved to {output_file}")
    else:
        print("No results to save. Extraction might have failed.")

In [27]:
folder_path = "/kaggle/input/indian-supreme-court-judgments/pdfs"
output_file = "/kaggle/working/output.csv"

start_time = time()
process_pdfs_in_folder(folder_path, output_file)
end_time = time()

print(f"Processing completed in {end_time - start_time:.2f} seconds.")

Processing PDFs: 100%|██████████| 48294/48294 [2:34:03<00:00,  5.22file/s]   


Extraction completed successfully. Results saved to /kaggle/working/output.csv
Processing completed in 9325.58 seconds.


In [7]:
import dask_cudf as dc

In [8]:
df = dc.read_csv("/kaggle/working/output.csv")

In [23]:
# Get the first partition as a cudf DataFrame and print the first value 
first_partition = df.partitions[0].compute() # Extract the value from "Extracted Text" 
te = first_partition["Extracted Text"]

In [31]:
def clean_text_column(col):
    col = col.str.lower()  # Convert to lowercase
    col = col.str.replace("\n", " ", regex=False)  # Replace newlines with spaces
    col = col.str.replace(r"http://judis\.nic\.in", "", regex=True)  # Remove specific substring
    col = col.str.replace(r"[^a-zA-Z0-9\s]", "", regex=True)  # Remove special characters
    col = col.str.replace(r"\s+", " ", regex=True)  # Replace multiple spaces with a single space
    col = col.str.strip()  # Strip leading/trailing spaces
    return col

In [32]:
df["Extracted Text"] = clean_text_column(df["Extracted Text"])

In [34]:
import pandas as pd
import re

# Load the dataset into a pandas DataFrame
df = pd.read_csv("/kaggle/working/output.csv")

In [35]:
# Define a function to clean and normalize text
def clean_text(text):
    if pd.isna(text):  # Check for NaN values
        return ""
    text = text.lower()  # Convert to lowercase
    text = text.replace("\n", " ")  # Replace newlines with spaces
    text = re.sub(r"http://judis\.nic\.in", "", text)  # Remove specific substring
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text)  # Remove special characters
    text = re.sub(r"\s+", " ", text)  # Replace multiple spaces with a single space
    text = text.strip()  # Strip leading/trailing spaces
    return text

In [36]:
# Apply the text cleaning function to the 'text_column'
df["Extracted Text"] = df["Extracted Text"].apply(clean_text)

In [37]:
# Write the cleaned data back to a CSV file
df.to_csv("cleaned_dataset.csv", index=False)

In [38]:
df["Extracted Text"][0]

'supreme court of india page 1 of 10 petitioner rural litigation and entitlementkendra ors etc vs respondent state of uttar pradesh ors etc date of judgment18121986 bench bhagwati pn cj bench bhagwati pn cj misra rangnath citation 1987 air 359 1987 scr 1 641 1986 scc supl 517 jt 1986 1119 1986 scale 21083 citator info rf 1987 sc2426 5 r 1988 sc2187 14 act constitution of india 1950 article 51agenvironmentpreservation ofecological balance keeping unaffectedtaskgovernment as also every citizen undertake mineral regulations act 1948minerals concession rules 1949 limestonequarrying ofgrant of mining leasesdepos its not to be exploited at cost of ecology and environmental consideration headnote a letter received from the rural litigation and entitle ment kendra dehradun was treated as a writ petition and notices issued the main allegations therein related to unauthorised and illegal mining operations carried on in the mussoorie hills and the area around adversely affecting the ecology of th

In [39]:
# Load the dataset into a Dask-cuDF DataFrame
df = dc.read_csv("/kaggle/working/cleaned_dataset.csv")

In [40]:
# Define a function to calculate the number of partitions needed
def calculate_partitions(df, max_file_size_mb, average_row_size_bytes=500):
    # Estimate the size of the DataFrame in bytes
    row_count = df.shape[0].compute()  # Total number of rows
    df_size_bytes = row_count * average_row_size_bytes
    df_size_mb = df_size_bytes / (1024 * 1024)  # Convert to MB

    # Calculate the number of partitions needed
    num_partitions = max(1, int(df_size_mb / max_file_size_mb) + 1)
    return num_partitions

In [41]:
# Set the maximum file size (200 MB)
MAX_FILE_SIZE_MB = 200
AVERAGE_ROW_SIZE_BYTES = 500  # Estimate the average row size in bytes

In [44]:
# Repartition the DataFrame based on the desired file size
num_partitions = calculate_partitions(df, MAX_FILE_SIZE_MB, AVERAGE_ROW_SIZE_BYTES)
print(num_partitions)

1


In [45]:
df = df.repartition(npartitions=5)

In [46]:
# Write the DataFrame to multiple CSV files
df.to_csv("cleaned_dataset_*.csv", index=False)

['/kaggle/working/cleaned_dataset_0.csv',
 '/kaggle/working/cleaned_dataset_1.csv',
 '/kaggle/working/cleaned_dataset_2.csv',
 '/kaggle/working/cleaned_dataset_3.csv',
 '/kaggle/working/cleaned_dataset_4.csv']

In [47]:
cl = pd.read_csv('/kaggle/working/cleaned_dataset_1.csv')
cl

,File Name,Extracted Text
0,5916-2005___supremecourt__2005__5916__5916_200...,1 reportable in the supreme court of india civ...
1,10146-1999___jonew__judis__20501.pdf,supreme court of india page 1 of 4 case no app...
2,22008-2002___jonew__judis__34738.pdf,reportable in the supreme court of india crimi...
3,70229-1989___jonew__judis__15566.pdf,supreme court of india page 1 of 3 petitioner ...
4,60214-1968___jonew__judis__5412.pdf,supreme court of india page 1 of 9 petitioner ...
...,...,...
9286,60613-1975___jonew__judis__9919.pdf,supreme court of india page 1 of 12 petitioner...
9287,1-1963___jonew__judis__3079.pdf,supreme court of india page 1 of 35 petitioner...
9288,60039-1968___jonew__judis__7264.pdf,supreme court of india page 1 of 19 petitioner...
9289,60234-1966___jonew__judis__2535.pdf,supreme court of india page 1 of 5 petitioner ...
